<a href="https://colab.research.google.com/github/ekaterina-kosti/Report/blob/main/Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Introduction*

In the modern age of high-throughput sequencing that produces an enormous amount of genomic data, typically stored in a FASTA format, it is sometimes practical to have a rapid characterisation and overview before more specialised analyses can be performed. Preliminary assessments are key in identifying key features of the data sets, filtering irrelevant data and guiding the direction of further analysis.

The following Python script is developed with the purpose of addressing such a need. Specifically, the program can read, parse and perform basic analysis on FASTA files containing DNA sequences.
Further functionalities include:
- Identifying and returning the length of the longest and shortest sequences in the given data set, which might be important for quality check of the sequencing.
- Finding Open Reading Frames (ORFs) and analysing them with respect to the reading frame, which is a crucial first step in understanding protein functionality.
- Finding conserved regions inside individual sequences and across the data set and analysing their relative frequencies, which provides insight into potential regulatory motifs, structural features or evolutionary distances between the sequences.

This program serves as a very simple utility for preliminary data assessment, to quickly identify sequences of interest or exclude data that does not meet specified criteria, optimising subsequent in-depth analyses. A more detailed overview of each functionality is provided below, in the relevant "*Code snippet description*" sections. The results, current limitations and possible solutions, as well as the potential for further development and expansions, are stated in the "*Results*" and "*Discussion*" sections, respectively.

# *Results and limitations*

The Python program was tested using the provided `dna2.fasta` example file to validate its core functionalities. The results demonstrate its capability to accurately parse FASTA data and perform the intended basic analyses:

- Sequence Overview: the program successfully identified 18 distinct DNA sequences. The longest sequence was found to be `gi|142022655|gb|EQ086233.1|255` with a length of 4894 base pairs (bp), while the shortest sequence was `gi|142022655|gb|EQ086233.1|346` with 115 bp.

- Open Reading Frame (ORF) Analysis: for a specified reading frame (e.g., frame 2), the program accurately identified ORFs. The longest ORF discovered across all sequences was [paste longest ORF sequence] with a length of 1821 bp, located within the sequence `gi|142022655|gb|EQ086233.1|527` and starting at position 636.

- Repeats Identification: when analysing 7-mer repeats, the program successfully identified and quantified recurring motifs. The most frequently occurring was "CGCGCCG", observed 63 times across the entire dataset.

Further results can be seen below the code cells.

The program completed these analyses on the `dna2.fasta` file (48 KB in size) with a runtime of 0.022 seconds, indicating its efficiency for smaller datasets. However, its' computational efficiency for larger data sets is yet to be determined and might be a potential area of future improvement. The time complexity of the program is not linear due to the implementation of nested loops and iterative functions. Its' memory may also require improvement, as it relies on storing sequences in a single dictionary, which might be problematic for heavy FASTA files. Therefore, improving computational efficiency, running time and memory use could be the next primary objective of improving the program.

### *Code snippet description:*

This chunk ofcode defines a function `creating_dict()` that reads a FASTA file from a specified path. It parses the DNA sequences within this file, creating a dictionary where sequence identifiers are keys and their corresponding DNA sequences are values.

After calling this function to populate the `seqs` dictionary, it is possible then to:
- Print the total number of entries (sequences) found.
- Extract all the DNA sequences into a list
- Sort this list of sequences by their length and identify shortest and longest sequences and their number.

In [1]:
from typing import Dict, List, Tuple

def creating_dict() -> Dict[str, str]:
    """
    Reads a FASTA file and parses DNA sequences into a dictionary.

    The dictionary keys are sequence identifiers and the values
    are the corresponding DNA sequences.

    Returns:
        Dict[str, str]: A dictionary where keys are sequence names and values are
                        the DNA sequences.
    """
    File_path = "dna2.fasta"
    f=open(File_path)
    seqs = {}
    for line in f:
        line = line.rstrip()
        if line[0] == ">":
            words = line.split()
            name = words[0][1:]  # Extract sequence name & removes '>'
            seqs[name] = ""  # Initialize sequence entry
        else:  # sequence, not header
            seqs[name] = seqs[name] + line
    f.close()
    return seqs

seqs = creating_dict()
#for name,seq in seqs.items():
#         print(name,seq)
print(f"Number of entries: {len(seqs)}") # number of entries

list_seqs = list(seqs.values()) # List of all dictionary values
sorted_list_seqs = sorted(list_seqs, key=len) # Sort by length

FileNotFoundError: [Errno 2] No such file or directory: 'dna2.fasta'

### *Code snippet description:*

With an existing dictionary of DNA sequences, it is now possible to perform further analysis. For example to identify and report on the longest and shortest sequences.


In [ ]:
list_seqs = list(seqs.values()) # List of all dictionary values
sorted_list_seqs = sorted(list_seqs, key=len) # Sort by length

list_of_longest_sequences = []
max_len = len(sorted_list_seqs[-1])  # Get the length of the longest sequence
print(f"The maximum sequence length is: {max_len}")

for name, seq in seqs.items():
    if len(seq) == max_len:  # Compare each sequence's length to the maximum
        list_of_longest_sequences.append(seq)

print(f"The longest sequence is {list(seqs.keys())[list(seqs.values()).index(list_of_longest_sequences[0])]}")

if len(list_of_longest_sequences) > 1:
    print("More than one longest sequence!")
    for seq in list_of_longest_sequences:
        print(list(seqs.keys())[list(seqs.values()).index(seq)]) # Print all identifiers
else:
    print("Only one longest sequence!")

list_of_shortest_sequences = []
min_len = len(sorted_list_seqs[0])  # Get the length of the shortest sequence
print(f"The minimum sequence length is: {min_len}")
for name, seq in seqs.items():
    if len(seq) == min_len:  # Compare each sequence's length to the minimum
        list_of_shortest_sequences.append(seq)

print(f"The shortest sequence is {list(seqs.keys())[list(seqs.values()).index(list_of_shortest_sequences[0])]}")

if len(list_of_shortest_sequences) > 1:
    print("More than one shortest sequence!")
    for seq in list_of_shortest_sequences:
        print(list(seqs.keys())[list(seqs.values()).index(seq)]) # print all identifiers
else:
    print("Only one shortest sequence!")

### *Code snippet description:*

Open reading frames define sections of DNA (or RNA) that can potentially encode proteins. Therefore, they always start with a start codon "ATG" and end with one of the stop codons "TGA", "TAG", "TAA". Identifying ORFS is a crucial step in genomic analysis.

The function, `new_dict_create()`, is designed to identify Open Reading Frames (ORFs) for a specified reading frame (1,2,3) represented by an integer (0,1,2).
* For each DNA sequence in the input seqs dictionary, it iterates through the sequence starting from the given frame and in steps of 3 (codon by codon).
* When a start_codon is encountered, it begins building an ORF by extending it with subsequent codons.
* This extension continues until one of the stop_codons is found, at which point the complete ORF is added to a list.

In [ ]:
def new_dict_create(seqs: Dict[str, str], frame: int) -> Dict[str, List[str]]:
    """
    Identifies Open Reading Frames (ORFs) within DNA sequences for a given reading frame.
    ORFs are found for each sequence in the input dictionary and stored in a new dictionary.

    Args:
        seqs (Dict[str, str]): A dictionary where keys are sequence identifiers and values are
                                DNA sequences.
        frame (int): The reading frame (0, 1, or 2).

    Returns:
        Dict[str, List[str]]: A dictionary where keys are sequence identifiers and values are
                              lists of ORFs found in that sequence for the specified frame.
    """
    start_codon = "ATG"
    stop_codons = ["TGA", "TAG", "TAA"]
    ORF_seqs = {}    # Initialize dictionary

    for name, seq in seqs.items():
        dna = seq
        ORF_seqs[name] = []
        ORF_line = []

        for i in range (frame, len(dna),3):
           codon = dna[i:i+3] # Define codon separation

           if codon == start_codon:
               ORF = codon
                 # Extend the ORF until a stop codon is found
               for j in range (i+3, len(dna), 3):
                   next_codon = dna[j:j+3]
                   ORF += next_codon # Extend each ORF

                   if next_codon in stop_codons:
                       ORF_line.append(ORF)
                       #print(f"ORF found: {ORF} in sequence {name}")


                       ORF_seqs[name] = ORF_line
                       break
    return ORF_seqs
ORF_seqs = new_dict_create(seqs, frame=2)

### *Code snippet description:*

To perform further analysis on identified ORFs, function `find_longest_ORF()` is called. It is designed to identify the single longest ORF from a collection of ORFs found in multiple DNA sequences.

It returns a tuple containing two strings:
   * The longest ORF sequence found across all input sequences.
   * The name of the sequence where that longest ORF was located.

This helps in pinpointing the most significant potential protein-coding region in a dataset.

In [ ]:
def find_longest_ORF(ORF_seqs) -> Tuple[str, str]:
    """
    Finds the longest Open Reading Frame (ORF) across all sequences and its
    corresponding sequence identifier.

    Args:
        ORF_seqs (Dict[str, List[str]]): A dictionary where keys are sequence identifiers
                                         and values are lists of ORFs.

    Returns:
        Tuple[str, str]: A tuple containing:
                         - The longest ORF sequence (str).
                         - The name of the sequence where the longest ORF was found (str).
    """
    list_orfs = list(ORF_seqs.values())# list of values, ie still lists
    longest_ORF = ""
    longest_ORF_sequence_name = ""
    for sequence_name, orf_list in ORF_seqs.items():  # Iterate through sequences and their ORFs
        for orf in orf_list:  # Iterate through ORFs in the current sequence
            if len(orf) > len(longest_ORF):  # Compare ORF lengths
                longest_ORF = orf
                longest_ORF_sequence_name = sequence_name

    return longest_ORF, longest_ORF_sequence_name

result = find_longest_ORF(ORF_seqs)
longest_orf_sequence = result[0]
longest_orf_sequence_name = result[1]

print("Longest ORF:", longest_orf_sequence,'\n', len(longest_orf_sequence))
print("Sequence name:", longest_orf_sequence_name)

### *Code snippet description:*

This section provides additional functionality to working with identified ORFs.

The function `find_sequence_by_ORF()` locates the original sequence identifier that contains a specific target ORF sequence. In case the provided ORF is not found or not viable, it returns `None`

The function `find_longest_ORF_by_ident()` identifies the longest ORF specifically within a single, designated sequence. If no ORFs are found for that identifier, it returns an empty string.

The function `starting_position()` uses `find_sequence_by_ORF` to get the key (identifier) of the original sequence containing the `longest_orf_sequence`.
It returns the int starting position (1-based) of the longest ORF within its original sequence, or None if for some reason it cannot be found. In principle any other sequence can be used and `longest_orf_sequence` was used as an example for demonstrativeness.

In [ ]:
def find_sequence_by_ORF(ORF_seqs: Dict[str, List[str]], target_value: str) -> str | None:
    """
    Finds the identifier of the sequence that contains a specific ORF.

    Args:
        ORF_seqs (Dict[str, List[str]]): A dictionary where keys are sequence identifiers
                                         and values are lists of ORFs.
        target_value (str): The ORF sequence to search for.

    Returns:
        Union[str, None]: The identifier of the sequence if the ORF is found,
                          otherwise None.
    """
    for key, value_list in ORF_seqs.items():
        if target_value in value_list:
            return key
    return None
print(find_sequence_by_ORF(ORF_seqs, target_value = longest_orf_sequence))

def find_longest_ORF_by_ident(ORF_seqs: Dict[str, List[str]], identifier: str) -> str:
        """
    Finds the longest ORF within a specific sequence identified by its identifier.

    Args:
        ORF_seqs (Dict[str, List[str]]): A dictionary where keys are sequence identifiers
                                         and values are lists of ORFs.
        identifier (str): The identifier of the sequence to search within.

    Returns:
        str: The longest ORF sequence found within the specified identifier.
             Returns an empty string if no ORFs are found for the identifier.
    """
        ORFs_in_seq = ORF_seqs[identifier]
        longest_ORF_in_seq = max(ORFs_in_seq, key=len)
        return longest_ORF_in_seq

print("The length of the longest ORF for this identifier is", len(find_longest_ORF_by_ident(ORF_seqs, "gi|142022655|gb|EQ086233.1|16")))

def starting_position(longest_orf_sequence: str, seqs: Dict[str, str]) -> int | None:
    """
    Finds the starting position of the longest ORF within its original sequence.

    Args:
        longest_orf_sequence (str): The longest ORF sequence.
        seqs (Dict[str, str]): The dictionary of all original DNA sequences.

    Returns:
        Union[int, None]: The starting position of the longest ORF if found,
                          otherwise None.
    """
    key = find_sequence_by_ORF(ORF_seqs, longest_orf_sequence)
    for name, seq in seqs.items():
        name = key
        seq = seqs[name]

        position = seq.find(longest_orf_sequence) +1

        return position
    return None
print(longest_orf_sequence)
start_pos = starting_position(longest_orf_sequence, seqs)
print(f"Starting position of the longest ORF: {start_pos}")

### *Code snippet description:*

Repetitive DNA sequences (repeats) are segments of DNA that re-occur within a genome. Repeats have defined functions, such as: regulation of gene expression, indication of mutations. The latter is why repeats' identification and annotation is an important part of bioinformatics research. This code provides a way to discover and quantify frequent short DNA sequence motifs.

The function `find_repeats()` identifies all unique substrings of a given length n that appear more than once within each individual DNA sequence
and stores them stores a list of these unique repeating substrings for each sequence identifier in the repeat_dict.

The function `count_repeats()` calculates the total occurrences of each unique repeating substring across all DNA sequences in the dataset.
It creates a new dictionary `count_repeats_dict` where keys are the unique repeating substrings, and values are their total frequencies across all DNA sequences.
In summary, this code provides a robust way to discover and quantify frequent short DNA sequence motifs within a collection of genetic data.

In [ ]:
def find_repeats(seqs: Dict[str, str], n: int) -> Dict[str, List[str]]:
    """
    Finds all repeats of length 'n' that appear more than once within each DNA sequence.

    Args:
        seqs (Dict[str, str]): A dictionary where keys are sequence identifiers and values are
                               DNA sequences.
        n (int): The length of the repeat to search for.

    Returns:
        Dict[str, List[str]]: A dictionary where keys are sequence identifiers and values are
                              lists of unique repeating substrings of length 'n' found
                              in that sequence.
    """
    repeat_dict = {}
    for name, seq in seqs.items():
        substring_count = {}
        repeat_set = set()
        for i in range(0, len(seq) -n+1):
            substring = seq[i:i+n]
            substring_count[substring] = substring_count.get(substring, 0)+1
        repeat_set = {sub for sub, count in substring_count.items() if count > 1}
        repeat_dict[name] = list(repeat_set)
    return repeat_dict

repeat_dict = find_repeats(seqs, 7)
#print(repeat_dict)



def count_repeats(repeat_dict: Dict[str, List[str]], seqs: Dict[str, str]) -> Dict[str, int]:
    """
    Counts the total occurrences of each unique repeat  across all DNA sequences in the file.

    Args:
        repeat_dict (Dict[str, List[str]]): A dictionary containing unique repeating substrings
                                             per sequence, as generated by `find_repeats`.
        seqs (Dict[str, str]): The dictionary of all original DNA sequences.

    Returns:
        Dict[str, int]: A dictionary where keys are the unique repeating substrings and
                        values are their total counts across all sequences.
    """
    count_repeats_dict = {}
    for _, repeats in repeat_dict.items():
        for r in repeats:
            total_count = 0
            for seq in seqs.values():
                total_count += seq.count(r)
            count_repeats_dict[r] = total_count
    return count_repeats_dict
repeat_dict = find_repeats(seqs, n=7)
count_repeats_dict = count_repeats(repeat_dict, seqs)
#
#
# print(count_repeats_dict)

repeat_dict = find_repeats(seqs, 7)
count_repeats_dict = count_repeats(repeat_dict, seqs)  # Pass dna_data
#print(count_repeats_dict)
sorted_repeat_frequency = sorted(list(count_repeats_dict.values()))
most_frequent_repeat = sorted_repeat_frequency[-1]
print(f"Highest frequency count for any repeat: {most_frequent_repeat}")

### *Code snippet description:*

Additionally, it might be necessary to identify which repeat is the most frequent for a given data-set. For example, it might indicate a common structural or functional pattern, or even evolutionary information in the context of sequence similarity.

The function, `find_most_frequent_repeat`, is designed to retrieve one of the DNA substrings that has the highest overall frequency from a dictionary of repeat counts.

In [ ]:
def find_most_frequent_repeat(count_repeats_dict: Dict[str, int], most_frequent_repeat: int) -> str | None:
    """
    Finds a repeat that has the highest frequency.

    Args:
        count_repeats_dict (Dict[str, int]): A dictionary where keys are repeating substrings
                                              and values are their total counts.
        most_frequent_repeat_count (int): The highest frequency value found in the dictionary.

    Returns:
        Union[str, None]: A repeat  with the highest frequency, or None if
                          the dictionary is empty or no such repeat is found.
    """
    for key, value in count_repeats_dict.items():
        if value == most_frequent_repeat:
            return key
    return None
#print(find_most_frequent_repeat(count_repeats_dict, most_frequent_repeat))
most_frequent_repeat_substring = find_most_frequent_repeat(count_repeats_dict, most_frequent_repeat)
if most_frequent_repeat_substring:
    print(f"One of the most frequent repeats is: {most_frequent_repeat_substring}")
else:
    print("No such repeat found.")

## Discussion

In essence, the project was successful in developing a Python script capable of performing a preliminary analysis of DNA sequences stored in a FASTA formatted file. The program is capable of parsing input files, calculating basic descriptive statistics such as the longest and shortest sequences, identifying Open Reading Frames (ORFs) across specified reading frames, and pinpointing recurring sequence motifs or repeats.

Furthermore, the functionalities presented in this report are not exhaustive, and several additions and improvements to the current version script are possible. For example, performance optimisation could include using and integrating specialised bioinformatics libraries (like Biopython), algorithmic refinement and memory management.Additional features could include GC content calculation, generation of reverse complement and transcription into RNA strands, translation into protein sequences, and creation of sequence logos for more useful insights.

## References

1. Library of Congress. (n.d.). FASTA format. Retrieved June 24, 2025, from https://www.loc.gov/preservation/digital/formats/fdd/fdd000622.shtml

2. Fraser, P., & Bickmore, W. A. (2018). Nuclear architecture and the genome. Trends in Genetics, 34(5), 341–352. https://doi.org/10.1016/j.tig.2017.12.009

3. Fischer, L., Böhm, L., Rall, K., Kreyenberg, H., Kunkel, H., Greve, J., & Sievers, M. (2023). Deep learning-based image analysis for predicting cell viability in ex vivo organoids. Communications Biology, 6(1), 939. https://doi.org/10.1038/s42003-023-05322-y

4. Wilf, H.S. (2002). Algorithms and Complexity (2nd ed.). A K Peters/CRC Press. https://doi.org/10.1201/9780429294921

5. Cock, P. J., Antao, T., Chang, J. T., Chapman, B. A., Cox, C. J., Dalke, A., … others. (2009). Biopython: freely available Python tools for computational molecular biology and bioinformatics. Bioinformatics, 25(11), 1422–1423.